# ERA5 Data Processing and Microsoft Aurora Weather Prediction for Maharashtra, India

This notebook demonstrates the complete workflow of:
1. Using the `eranest` package to download ERA5 data for Maharashtra, India
2. Processing the data into Aurora-compatible format
3. Running Microsoft Aurora weather predictions
4. Visualizing the results

## About Maharashtra
Maharashtra is a state in western India with Mumbai as its capital. It's the second-most populous state in India and covers an area of approximately 307,713 km².

---

## Prerequisites

Make sure you have:
- CDS API credentials configured (see `eranest` documentation)
- Required Python packages installed
- Access to Microsoft Aurora model (via Hugging Face or local installation)

In [1]:
# Import required libraries
import sys
import os
import json
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📦 Libraries imported successfully!")

📦 Libraries imported successfully!


In [2]:
# Import eranest package components
try:
    from eranest.core2 import (
        era5_aurora_wrapper,
        AURORA_PRESSURE_LEVELS,
        SURFACE_VARIABLES,
        ATMOSPHERIC_VARIABLES,
        STATIC_VARIABLES
    )
    print("✅ eranest package imported successfully!")
    print(f"📊 Available Aurora pressure levels: {AURORA_PRESSURE_LEVELS}")
    print(f"🌡️  Surface variables: {SURFACE_VARIABLES}")
    print(f"🌪️  Atmospheric variables: {ATMOSPHERIC_VARIABLES}")
    print(f"🗺️  Static variables: {STATIC_VARIABLES}")
except ImportError as e:
    print(f"❌ Error importing eranest: {e}")
    print("Make sure eranest is installed and in your Python path")

✅ eranest package imported successfully!
📊 Available Aurora pressure levels: ['50', '100', '150', '200', '250', '300', '400', '500', '600', '700', '850', '925', '1000']
🌡️  Surface variables: ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure']
🌪️  Atmospheric variables: ['temperature', 'u_component_of_wind', 'v_component_of_wind', 'specific_humidity', 'geopotential']
🗺️  Static variables: ['geopotential', 'land_sea_mask', 'soil_type']


## 1. Setting Up Maharashtra Boundary Data

We'll load the Maharashtra state boundary from the provided India GeoJSON file.

In [3]:
# Load Maharashtra boundary
maharashtra_geojson_path = '../maharashtra_boundary.geojson'

# Check if file exists, if not create it from the India GeoJSON
if not os.path.exists(maharashtra_geojson_path):
    print("📍 Extracting Maharashtra boundary from India GeoJSON...")
    
    # Load the large India GeoJSON file
    india_geojson_path = '/Users/saket/github/india-covid-explorer-viz/public/LGD_States_smooth_optimized.geojson'
    
    try:
        with open(india_geojson_path, 'r') as f:
            india_data = json.load(f)
        
        # Find Maharashtra feature
        maharashtra_feature = None
        for feature in india_data['features']:
            if ('properties' in feature and 
                'state_name' in feature['properties'] and 
                feature['properties']['state_name'] == 'Maharashtra'):
                maharashtra_feature = feature
                break
        
        if maharashtra_feature:
            # Create new GeoJSON with just Maharashtra
            maharashtra_geojson = {
                'type': 'FeatureCollection',
                'features': [maharashtra_feature]
            }
            
            # Save to file
            with open(maharashtra_geojson_path, 'w') as f:
                json.dump(maharashtra_geojson, f, indent=2)
            
            print("✅ Maharashtra boundary extracted and saved!")
        else:
            print("❌ Maharashtra not found in the India GeoJSON file")
            
    except FileNotFoundError:
        print(f"❌ India GeoJSON file not found at: {india_geojson_path}")
        print("Using a simple bounding box for Maharashtra instead...")
        
        # Create a simple bounding box for Maharashtra
        maharashtra_bbox = {
            "type": "FeatureCollection",
            "features": [{
                "type": "Feature",
                "properties": {
                    "state_name": "Maharashtra",
                    "description": "Approximate bounding box for Maharashtra"
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[
                        [72.6, 15.6],  # Southwest corner
                        [80.9, 15.6],  # Southeast corner
                        [80.9, 22.0],  # Northeast corner
                        [72.6, 22.0],  # Northwest corner
                        [72.6, 15.6]   # Close the polygon
                    ]]
                }
            }]
        }
        
        with open(maharashtra_geojson_path, 'w') as f:
            json.dump(maharashtra_bbox, f, indent=2)
        
        print("✅ Maharashtra bounding box created!")

# Load and display Maharashtra boundary info
with open(maharashtra_geojson_path, 'r') as f:
    maharashtra_data = json.load(f)

print(f"🗺️  Maharashtra GeoJSON loaded successfully!")
print(f"📐 Number of features: {len(maharashtra_data['features'])}")
print(f"🏷️  Properties: {list(maharashtra_data['features'][0]['properties'].keys())}")

🗺️  Maharashtra GeoJSON loaded successfully!
📐 Number of features: 1
🏷️  Properties: ['state_name', 'state_code']


## 2. Configure ERA5 Data Download Parameters

We'll set up the parameters for downloading ERA5 data using the eranest package. This includes:
- Date range for historical data (needed for Aurora initialization)
- Variables selection (surface, atmospheric, and static)
- Spatial resolution

In [4]:
# Configure download parameters
config = {
    'request_id': 'maharashtra_aurora_demo',
    'start_date': dt.datetime(2024, 1, 1, 0, 0),    # Start with recent data
    'end_date': dt.datetime(2024, 1, 2, 0, 0),      # Two days for Aurora (needs current + previous)
    'resolution': 0.25,  # 0.25 degree resolution (about 25km)
    'frequency': 'hourly',
    
    # Aurora-compatible variables
    'surface_variables': [
        '2m_temperature',                    # 2t in Aurora
        '10m_u_component_of_wind',          # 10u in Aurora  
        '10m_v_component_of_wind',          # 10v in Aurora
        'mean_sea_level_pressure'           # msl in Aurora
    ],
    
    'atmospheric_variables': [
        'temperature',                       # t in Aurora
        'u_component_of_wind',              # u in Aurora
        'v_component_of_wind',              # v in Aurora
        'specific_humidity',                # q in Aurora
        'geopotential'                      # z in Aurora
    ],
    
    'static_variables': [
        'geopotential',                     # z in Aurora (surface)
        'land_sea_mask',                    # lsm in Aurora
        'soil_type'                         # slt in Aurora
    ],
    
    # Use Aurora standard pressure levels
    'pressure_levels': AURORA_PRESSURE_LEVELS,
    
    'include_static': True
}

print("⚙️  ERA5 Download Configuration:")
print(f"📅 Date range: {config['start_date'].strftime('%Y-%m-%d')} to {config['end_date'].strftime('%Y-%m-%d')}")
print(f"🎯 Resolution: {config['resolution']}° (~{config['resolution']*111:.0f}km)")
print(f"⏰ Frequency: {config['frequency']}")
print(f"🌡️  Surface variables ({len(config['surface_variables'])}): {', '.join(config['surface_variables'])}")
print(f"🌪️  Atmospheric variables ({len(config['atmospheric_variables'])}): {', '.join(config['atmospheric_variables'])}")
print(f"🗺️  Static variables ({len(config['static_variables'])}): {', '.join(config['static_variables'])}")
print(f"📊 Pressure levels ({len(config['pressure_levels'])}): {', '.join(config['pressure_levels'])} hPa")

⚙️  ERA5 Download Configuration:
📅 Date range: 2024-01-01 to 2024-01-02
🎯 Resolution: 0.25° (~28km)
⏰ Frequency: hourly
🌡️  Surface variables (4): 2m_temperature, 10m_u_component_of_wind, 10m_v_component_of_wind, mean_sea_level_pressure
🌪️  Atmospheric variables (5): temperature, u_component_of_wind, v_component_of_wind, specific_humidity, geopotential
🗺️  Static variables (3): geopotential, land_sea_mask, soil_type
📊 Pressure levels (13): 50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000 hPa


## 3. Download ERA5 Data using eranest

Now we'll use the eranest package to download ERA5 data for Maharashtra. The `era5_aurora_wrapper` function will handle:
- Downloading surface-level, atmospheric, and static variables
- Filtering data to Maharashtra boundaries
- Formatting data for Aurora compatibility

⚠️ **Note**: This step requires CDS API credentials and may take several minutes to complete.

In [5]:
# Download ERA5 data for Maharashtra
print("🔄 Starting ERA5 data download for Maharashtra...")
print("This may take several minutes depending on data size and CDS queue.")
print("☕ Time for a coffee break!\n")

try:
    # Use eranest to download and process ERA5 data
    era5_data = era5_aurora_wrapper(
        request_id=config['request_id'],
        start_date=config['start_date'],
        end_date=config['end_date'],
        json_file=maharashtra_geojson_path,
        surface_variables=config['surface_variables'],
        atmospheric_variables=config['atmospheric_variables'],
        static_variables=config['static_variables'],
        pressure_levels=config['pressure_levels'],
        resolution=config['resolution'],
        frequency=config['frequency'],
        include_static=config['include_static']
    )
    
    print("\n✅ ERA5 data download completed successfully!")
    print(f"📦 Available data types: {list(era5_data.keys())}")
    
    # Display data summary
    for data_type, df in era5_data.items():
        print(f"\n📊 {data_type.upper()} DATA:")
        print(f"   Shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        print(f"   Date range: {df['time'].min()} to {df['time'].max()}" if 'time' in df.columns else "   No time dimension")
        print(f"   Unique locations: {len(df[['latitude', 'longitude']].drop_duplicates())}")
        
except Exception as e:
    print(f"❌ Error downloading ERA5 data: {e}")
    print("\n🔧 Troubleshooting tips:")
    print("1. Check your CDS API credentials (~/.cdsapirc)")
    print("2. Verify internet connection")
    print("3. Check CDS service status")
    print("4. Try reducing date range or spatial coverage")
    
    # For demo purposes, create mock data
    print("\n🎭 Creating mock data for demonstration...")
    era5_data = create_mock_era5_data(config)

🔄 Starting ERA5 data download for Maharashtra...
This may take several minutes depending on data size and CDS queue.
☕ Time for a coffee break!

Successfully loaded JSON file with utf-8 encoding
Valid GeoJSON detected: ../maharashtra_boundary.geojson
✓ CDS API configuration is already set up and valid.

STARTING ERA5 AURORA DATA PROCESSING
Request ID: maharashtra_aurora_demo
Date Range: 2024-01-01 to 2024-01-02
Frequency: hourly
Resolution: 0.25°
GeoJSON File: ../maharashtra_boundary.geojson
Surface variables: ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure']
Atmospheric variables: ['temperature', 'u_component_of_wind', 'v_component_of_wind', 'specific_humidity', 'geopotential']
Static variables: ['geopotential', 'land_sea_mask', 'soil_type']
Pressure levels: ['50', '100', '150', '200', '250', '300', '400', '500', '600', '700', '850', '925', '1000']

--- Input Validation ---
✓ All inputs validated successfully

--- Loading GeoJSON File 

2025-06-19 16:57:43,985 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-19 16:57:43,987 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-19 16:57:43,988 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-19 16:57:44,844 INFO Request ID is 6459b89b-edb7-4b5d-9fa7-4098443c77c7
2025-06-19 16:57:45,078 INFO status has been updated to accepted
2025-06-19 16:57:50,559 INFO status has been updated to running
2025-06-19 16:58:19,274 INFO status has been updated to successful
                                                                                                            

Static variables download complete: maharashtra_aurora_demo_static.nc
✓ Static variables downloaded successfully

--- Downloading Surface-Level Variables ---
Date range: 2024-01-01 to 2024-01-02
Variables: 2m_temperature, 10m_u_component_of_wind, 10m_v_component_of_wind, mean_sea_level_pressure
Area: North: 22.0303, West: 72.642, South: 15.6061, East: 80.8977


2025-06-19 16:58:22,680 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-19 16:58:22,681 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-19 16:58:22,681 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-19 16:58:23,767 INFO Request ID is a0f7f219-43ed-46cf-9fd9-efb621765a77
2025-06-19 16:58:23,972 INFO status has been updated to accepted
2025-06-19 16:58:33,495 INFO status has been updated to running
2025-06-19 16:58:46,600 INFO status has been updated to successful
                                                                                                            

Surface-level download complete: maharashtra_aurora_demo_surface.nc
✓ Surface-level variables downloaded successfully

--- Downloading Atmospheric Variables ---
Date range: 2024-01-01 to 2024-01-02
Variables: temperature, u_component_of_wind, v_component_of_wind, specific_humidity, geopotential
Pressure levels: 50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000
Area: North: 22.0303, West: 72.642, South: 15.6061, East: 80.8977


2025-06-19 16:58:49,754 INFO [2025-06-16T00:00:00] CC-BY licence to replace Licence to use Copernicus Products on 02 July 2025. More information available [here](https://forum.ecmwf.int/t/cc-by-licence-to-replace-licence-to-use-copernicus-products-on-02-july-2025/13464)
2025-06-19 16:58:49,756 INFO [2025-06-10T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-06-19 16:58:49,757 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-19 16:58:50,263 INFO Request ID is 0e5548f3-580c-4c30-9ae8-30a28d458f51
2025-06-19 16:58:50,490 INFO status has been updated to accepted
2025-06-19 16:58:56,220 INFO status has been updated to running
2025-06-19 17:03:12,391 INFO status has been updated to successful
                                                                                                            

Atmospheric data download complete: maharashtra_aurora_demo_atmospheric.nc
✓ Atmospheric variables downloaded successfully

--- Processing Downloaded Files ---

→ Processing static data: maharashtra_aurora_demo_static.nc
  ✓ Loaded dataset with dimensions: {'valid_time': 1, 'latitude': 26, 'longitude': 34}
  → Filtering static data by shapefile...
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 884 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.06 seconds
  - Points inside: 428
  - Points outside: 456
  - Percentage inside: 48.42%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 884 rows
  ✓ Created lookup set with 428 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 884 to 428 rows
✓ Dataset filtering completed in 0.01 seconds

--- Final Filtering Results ---
Total proce

## 4. Data Exploration and Visualization

Let's explore the downloaded ERA5 data and visualize key meteorological variables over Maharashtra.

In [6]:
# Explore the downloaded data
print("🔍 EXPLORING ERA5 DATA FOR MAHARASHTRA")
print("=" * 50)

# Assume we have data from previous cell (either real or mock)
if 'era5_data' not in locals():
    print("Creating mock data for exploration...")
    era5_data = create_mock_era5_data(config)

# Surface data exploration
if 'surface' in era5_data:
    surface_df = era5_data['surface']
    print(f"\n🌡️  SURFACE DATA SUMMARY:")
    print(f"   📏 Shape: {surface_df.shape}")
    print(f"   📅 Time range: {surface_df['time'].min()} to {surface_df['time'].max()}")
    print(f"   🗺️  Spatial coverage: {len(surface_df[['latitude', 'longitude']].drop_duplicates())} grid points")
    print(f"   📊 Variables: {[col for col in surface_df.columns if col not in ['time', 'latitude', 'longitude']]}")
    
    # Basic statistics
    print(f"\n   📈 SURFACE VARIABLE STATISTICS:")
    numeric_cols = surface_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col not in ['latitude', 'longitude']:
            mean_val = surface_df[col].mean()
            std_val = surface_df[col].std()
            min_val = surface_df[col].min()
            max_val = surface_df[col].max()
            print(f"      {col}: {mean_val:.2f} ± {std_val:.2f} (range: {min_val:.2f} to {max_val:.2f})")

# Atmospheric data exploration
if 'atmospheric' in era5_data:
    atmos_df = era5_data['atmospheric']
    print(f"\n🌪️  ATMOSPHERIC DATA SUMMARY:")
    print(f"   📏 Shape: {atmos_df.shape}")
    print(f"   📊 Pressure levels: {sorted(atmos_df['level'].unique()) if 'level' in atmos_df.columns else 'Not available'}")
    print(f"   📊 Variables: {[col for col in atmos_df.columns if col not in ['time', 'latitude', 'longitude', 'level']]}")

# Static data exploration
if 'static' in era5_data:
    static_df = era5_data['static']
    print(f"\n🗺️  STATIC DATA SUMMARY:")
    print(f"   📏 Shape: {static_df.shape}")
    print(f"   📊 Variables: {[col for col in static_df.columns if col not in ['latitude', 'longitude']]}")
    
    # Land-sea mask analysis
    if 'lsm' in static_df.columns:
        land_points = (static_df['lsm'] > 0.5).sum()
        total_points = len(static_df)
        print(f"   🏞️  Land coverage: {land_points}/{total_points} points ({land_points/total_points*100:.1f}%)")

print("\n✅ Data exploration completed!")

🔍 EXPLORING ERA5 DATA FOR MAHARASHTRA

🌡️  SURFACE DATA SUMMARY:
   📏 Shape: (20544, 9)


KeyError: 'time'

In [ ]:
# Create visualizations of ERA5 data
print("📊 Creating visualizations of ERA5 data over Maharashtra...")

# Set up the plotting environment
plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['font.size'] = 10

# Create subplots for different variables
fig = plt.figure(figsize=(20, 15))

# Assume we have surface data
if 'surface' in era5_data:
    surface_df = era5_data['surface']
    
    # Get latest time step for spatial plots
    latest_time = surface_df['time'].max()
    latest_data = surface_df[surface_df['time'] == latest_time].copy()
    
    # Temperature map
    ax1 = plt.subplot(2, 3, 1, projection=ccrs.PlateCarree())
    ax1.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
    ax1.add_feature(cfeature.COASTLINE)
    ax1.add_feature(cfeature.BORDERS)
    ax1.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
    
    # Plot temperature
    temp_var = 't2m' if 't2m' in latest_data.columns else [col for col in latest_data.columns if 'temp' in col.lower()][0]
    scatter = ax1.scatter(latest_data['longitude'], latest_data['latitude'], 
                         c=latest_data[temp_var], cmap='RdYlBu_r', s=30, 
                         transform=ccrs.PlateCarree())
    ax1.set_title(f'Temperature (°C)\n{latest_time.strftime("%Y-%m-%d %H:%M UTC")}', fontsize=12)
    plt.colorbar(scatter, ax=ax1, shrink=0.7)
    ax1.gridlines(draw_labels=True, alpha=0.3)
    
    # Wind map
    ax2 = plt.subplot(2, 3, 2, projection=ccrs.PlateCarree())
    ax2.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
    ax2.add_feature(cfeature.COASTLINE)
    ax2.add_feature(cfeature.BORDERS)
    ax2.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
    
    # Plot wind vectors
    u_var = 'u10' if 'u10' in latest_data.columns else [col for col in latest_data.columns if 'u' in col.lower()][0]
    v_var = 'v10' if 'v10' in latest_data.columns else [col for col in latest_data.columns if 'v' in col.lower()][0]
    
    wind_speed = np.sqrt(latest_data[u_var]**2 + latest_data[v_var]**2)
    quiver = ax2.quiver(latest_data['longitude'], latest_data['latitude'],
                       latest_data[u_var], latest_data[v_var],
                       wind_speed, cmap='viridis', scale=50,
                       transform=ccrs.PlateCarree())
    ax2.set_title(f'Wind Vectors (m/s)\n{latest_time.strftime("%Y-%m-%d %H:%M UTC")}', fontsize=12)
    plt.colorbar(quiver, ax=ax2, shrink=0.7, label='Wind Speed (m/s)')
    ax2.gridlines(draw_labels=True, alpha=0.3)
    
    # Pressure map
    ax3 = plt.subplot(2, 3, 3, projection=ccrs.PlateCarree())
    ax3.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
    ax3.add_feature(cfeature.COASTLINE)
    ax3.add_feature(cfeature.BORDERS)
    ax3.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
    
    # Plot pressure
    press_var = 'msl' if 'msl' in latest_data.columns else [col for col in latest_data.columns if 'press' in col.lower()][0]
    pressure_scatter = ax3.scatter(latest_data['longitude'], latest_data['latitude'],
                                  c=latest_data[press_var]/100, cmap='RdBu_r', s=30,  # Convert to hPa
                                  transform=ccrs.PlateCarree())
    ax3.set_title(f'Sea Level Pressure (hPa)\n{latest_time.strftime("%Y-%m-%d %H:%M UTC")}', fontsize=12)
    plt.colorbar(pressure_scatter, ax=ax3, shrink=0.7)
    ax3.gridlines(draw_labels=True, alpha=0.3)
    
    # Time series plot
    ax4 = plt.subplot(2, 3, 4)
    
    # Average temperature over Maharashtra
    temp_ts = surface_df.groupby('time')[temp_var].mean()
    ax4.plot(temp_ts.index, temp_ts.values, 'r-', linewidth=2, label='Temperature')
    ax4.set_ylabel('Temperature (°C)', color='r')
    ax4.tick_params(axis='y', labelcolor='r')
    ax4.set_title('Average Meteorological Variables\nOver Maharashtra', fontsize=12)
    ax4.grid(True, alpha=0.3)
    
    # Add pressure on secondary axis
    ax4_twin = ax4.twinx()
    press_ts = surface_df.groupby('time')[press_var].mean()
    ax4_twin.plot(press_ts.index, press_ts.values/100, 'b-', linewidth=2, label='Pressure')
    ax4_twin.set_ylabel('Pressure (hPa)', color='b')
    ax4_twin.tick_params(axis='y', labelcolor='b')
    
    plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)
    
    # Wind speed distribution
    ax5 = plt.subplot(2, 3, 5)
    wind_speed_all = np.sqrt(surface_df[u_var]**2 + surface_df[v_var]**2)
    ax5.hist(wind_speed_all, bins=30, alpha=0.7, color='green', edgecolor='black')
    ax5.set_xlabel('Wind Speed (m/s)')
    ax5.set_ylabel('Frequency')
    ax5.set_title('Wind Speed Distribution\nOver Maharashtra', fontsize=12)
    ax5.grid(True, alpha=0.3)
    
    # Add statistics
    mean_wind = wind_speed_all.mean()
    max_wind = wind_speed_all.max()
    ax5.axvline(mean_wind, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_wind:.1f} m/s')
    ax5.legend()

# Static data visualization
if 'static' in era5_data:
    static_df = era5_data['static']
    
    ax6 = plt.subplot(2, 3, 6, projection=ccrs.PlateCarree())
    ax6.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
    ax6.add_feature(cfeature.COASTLINE)
    ax6.add_feature(cfeature.BORDERS)
    ax6.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
    
    # Plot topography
    topo_var = 'z' if 'z' in static_df.columns else [col for col in static_df.columns if col not in ['latitude', 'longitude', 'lsm', 'slt']][0]
    topo_scatter = ax6.scatter(static_df['longitude'], static_df['latitude'],
                              c=static_df[topo_var]/9.81, cmap='terrain', s=30,  # Convert to height
                              transform=ccrs.PlateCarree())
    ax6.set_title('Topography (m)\n(Surface Geopotential Height)', fontsize=12)
    plt.colorbar(topo_scatter, ax=ax6, shrink=0.7)
    ax6.gridlines(draw_labels=True, alpha=0.3)

plt.tight_layout()
plt.suptitle('ERA5 Meteorological Data Analysis for Maharashtra, India', fontsize=16, y=0.98)
plt.show()

print("✅ ERA5 data visualization completed!")

## 5. Prepare Data for Microsoft Aurora

Now we'll convert the ERA5 data into the format required by Microsoft Aurora. According to the Aurora documentation, we need to create a `Batch` object with:
- Surface variables (2t, 10u, 10v, msl)
- Static variables (lsm, z, slt)
- Atmospheric variables (z, u, v, t, q) at pressure levels
- Metadata (lat, lon, time, pressure levels)

In [ ]:
# Install Microsoft Aurora (if not already installed)
print("🔧 Setting up Microsoft Aurora...")

try:
    import torch
    print("✅ PyTorch available")
except ImportError:
    print("❌ PyTorch not found. Installing...")
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Try to import Aurora
try:
    from aurora import Batch, Metadata
    print("✅ Microsoft Aurora imported successfully!")
    AURORA_AVAILABLE = True
except ImportError:
    print("⚠️  Microsoft Aurora not available. Installing from source...")
    AURORA_AVAILABLE = False
    
    # For demo purposes, we'll create mock Aurora classes
    print("🎭 Creating mock Aurora classes for demonstration...")
    
    class MockMetadata:
        def __init__(self, lat, lon, time, atmos_levels):
            self.lat = lat
            self.lon = lon
            self.time = time
            self.atmos_levels = atmos_levels
    
    class MockBatch:
        def __init__(self, surf_vars, static_vars, atmos_vars, metadata):
            self.surf_vars = surf_vars
            self.static_vars = static_vars
            self.atmos_vars = atmos_vars
            self.metadata = metadata
            
        def __repr__(self):
            return f"MockBatch(surf_vars={list(self.surf_vars.keys())}, static_vars={list(self.static_vars.keys())}, atmos_vars={list(self.atmos_vars.keys())})"
    
    Batch = MockBatch
    Metadata = MockMetadata

print(f"🎯 Aurora setup complete (Available: {AURORA_AVAILABLE})")

In [ ]:
# Convert ERA5 data to Aurora format
print("🔄 Converting ERA5 data to Aurora-compatible format...")

def era5_to_aurora_batch(era5_data, config):
    """Convert ERA5 data to Aurora Batch format"""
    import torch
    from datetime import datetime
    
    print("📦 Processing ERA5 data for Aurora...")
    
    # Get surface data
    surface_df = era5_data.get('surface')
    static_df = era5_data.get('static')
    atmos_df = era5_data.get('atmospheric')
    
    if surface_df is None:
        raise ValueError("Surface data is required for Aurora")
    
    # Get unique coordinates and times
    coords = surface_df[['latitude', 'longitude']].drop_duplicates().sort_values(['latitude', 'longitude'])
    times = sorted(surface_df['time'].unique())
    
    print(f"   📍 Grid points: {len(coords)}")
    print(f"   ⏰ Time steps: {len(times)}")
    
    # Create coordinate arrays
    lats = torch.tensor(sorted(coords['latitude'].unique(), reverse=True), dtype=torch.float32)  # Aurora wants decreasing
    lons = torch.tensor(sorted(coords['longitude'].unique()), dtype=torch.float32)
    
    h, w = len(lats), len(lons)
    t = min(len(times), 2)  # Aurora needs current + previous step
    b = 1  # Batch size
    
    print(f"   📐 Grid dimensions: {h} x {w} (lat x lon)")
    print(f"   ⏳ Time history: {t} steps")
    
    # Initialize tensors
    surf_vars = {}
    static_vars = {}
    atmos_vars = {}
    
    # Process surface variables
    print("   🌡️  Processing surface variables...")
    
    # Variable name mapping from ERA5 to Aurora
    surf_var_mapping = {
        't2m': '2t',  # Temperature
        'u10': '10u',  # U-wind
        'v10': '10v',  # V-wind  
        'msl': 'msl'   # Mean sea level pressure
    }
    
    for era5_var, aurora_var in surf_var_mapping.items():
        if era5_var in surface_df.columns:
            # Create tensor (b, t, h, w)
            var_tensor = torch.zeros(b, t, h, w, dtype=torch.float32)
            
            for time_idx, time_val in enumerate(times[-t:]):  # Use last t time steps
                time_data = surface_df[surface_df['time'] == time_val]
                
                for _, row in time_data.iterrows():
                    lat_idx = torch.argmin(torch.abs(lats - row['latitude']))
                    lon_idx = torch.argmin(torch.abs(lons - row['longitude']))
                    var_tensor[0, time_idx, lat_idx, lon_idx] = row[era5_var]
            
            surf_vars[aurora_var] = var_tensor
            print(f"      ✓ {aurora_var}: {var_tensor.shape}")
    
    # Process static variables
    if static_df is not None:
        print("   🗺️  Processing static variables...")
        
        static_var_mapping = {
            'z': 'z',      # Surface geopotential
            'lsm': 'lsm',  # Land-sea mask
            'slt': 'slt'   # Soil type
        }
        
        for era5_var, aurora_var in static_var_mapping.items():
            if era5_var in static_df.columns:
                # Create tensor (h, w)
                var_tensor = torch.zeros(h, w, dtype=torch.float32)
                
                for _, row in static_df.iterrows():
                    lat_idx = torch.argmin(torch.abs(lats - row['latitude']))
                    lon_idx = torch.argmin(torch.abs(lons - row['longitude']))
                    var_tensor[lat_idx, lon_idx] = row[era5_var]
                
                static_vars[aurora_var] = var_tensor
                print(f"      ✓ {aurora_var}: {var_tensor.shape}")
    
    # Process atmospheric variables
    if atmos_df is not None and 'level' in atmos_df.columns:
        print("   🌪️  Processing atmospheric variables...")
        
        pressure_levels = sorted(atmos_df['level'].unique())
        c = len(pressure_levels)
        
        atmos_var_mapping = {
            't': 't',    # Temperature
            'u': 'u',    # U-wind
            'v': 'v',    # V-wind
            'q': 'q',    # Specific humidity
            'z': 'z'     # Geopotential
        }
        
        for era5_var, aurora_var in atmos_var_mapping.items():
            if era5_var in atmos_df.columns:
                # Create tensor (b, t, c, h, w)
                var_tensor = torch.zeros(b, t, c, h, w, dtype=torch.float32)
                
                for time_idx, time_val in enumerate(times[-t:]):
                    time_data = atmos_df[atmos_df['time'] == time_val]
                    
                    for level_idx, level in enumerate(pressure_levels):
                        level_data = time_data[time_data['level'] == level]
                        
                        for _, row in level_data.iterrows():
                            lat_idx = torch.argmin(torch.abs(lats - row['latitude']))
                            lon_idx = torch.argmin(torch.abs(lons - row['longitude']))
                            var_tensor[0, time_idx, level_idx, lat_idx, lon_idx] = row[era5_var]
                
                atmos_vars[aurora_var] = var_tensor
                print(f"      ✓ {aurora_var}: {var_tensor.shape}")
    
    # Create metadata
    pressure_levels_hpa = tuple(pressure_levels) if 'atmos_df' in locals() and atmos_df is not None else (850, 500, 250)
    current_time = times[-1] if times else datetime(2024, 1, 1, 12, 0)
    
    metadata = Metadata(
        lat=lats,
        lon=lons,
        time=(current_time,),
        atmos_levels=pressure_levels_hpa
    )
    
    # Create Aurora batch
    aurora_batch = Batch(
        surf_vars=surf_vars,
        static_vars=static_vars,
        atmos_vars=atmos_vars,
        metadata=metadata
    )
    
    print("\n✅ Aurora batch created successfully!")
    print(f"   📦 Surface variables: {list(surf_vars.keys())}")
    print(f"   🗺️  Static variables: {list(static_vars.keys())}")
    print(f"   🌪️  Atmospheric variables: {list(atmos_vars.keys())}")
    print(f"   📊 Pressure levels: {pressure_levels_hpa}")
    print(f"   📅 Current time: {current_time}")
    
    return aurora_batch

# Convert data to Aurora format
try:
    aurora_batch = era5_to_aurora_batch(era5_data, config)
    print(f"\n🎯 Aurora batch ready: {aurora_batch}")
except Exception as e:
    print(f"❌ Error creating Aurora batch: {e}")
    print("Creating a minimal mock batch for demonstration...")
    
    # Create mock tensors for demonstration
    import torch
    aurora_batch = Batch(
        surf_vars={'2t': torch.randn(1, 2, 10, 15), '10u': torch.randn(1, 2, 10, 15), 
                  '10v': torch.randn(1, 2, 10, 15), 'msl': torch.randn(1, 2, 10, 15)},
        static_vars={'lsm': torch.randn(10, 15), 'z': torch.randn(10, 15), 'slt': torch.randn(10, 15)},
        atmos_vars={'t': torch.randn(1, 2, 3, 10, 15), 'u': torch.randn(1, 2, 3, 10, 15),
                   'v': torch.randn(1, 2, 3, 10, 15), 'q': torch.randn(1, 2, 3, 10, 15),
                   'z': torch.randn(1, 2, 3, 10, 15)},
        metadata=Metadata(
            lat=torch.linspace(22.0, 15.6, 10),
            lon=torch.linspace(72.6, 80.9, 15),
            time=(dt.datetime(2024, 1, 1, 12, 0),),
            atmos_levels=(850, 500, 250)
        )
    )

## 6. Load and Run Microsoft Aurora Model

Now we'll load the Microsoft Aurora model and use it to generate weather predictions for Maharashtra. Aurora can predict weather conditions up to 10 days in advance.

In [ ]:
# Load Microsoft Aurora model
print("🤖 Loading Microsoft Aurora weather prediction model...")

def load_aurora_model():
    """Load Microsoft Aurora model"""
    if AURORA_AVAILABLE:
        try:
            # Try to load Aurora model from Hugging Face
            from aurora import Aurora
            
            print("   📥 Downloading Aurora model from Hugging Face...")
            model = Aurora(use_lora=False)  # Use full model, not LoRA
            model.eval()  # Set to evaluation mode
            
            print("   ✅ Aurora model loaded successfully!")
            return model, True
            
        except Exception as e:
            print(f"   ⚠️  Error loading real Aurora model: {e}")
            print("   🎭 Creating mock model for demonstration...")
            return create_mock_aurora_model(), False
    else:
        print("   🎭 Aurora not available, creating mock model...")
        return create_mock_aurora_model(), False

def create_mock_aurora_model():
    """Create a mock Aurora model for demonstration"""
    class MockAurora:
        def __init__(self):
            self.name = "Mock Aurora Model"
        
        def forward(self, batch):
            """Mock forward pass that returns modified input"""
            import torch
            import copy
            
            # Create a mock prediction by modifying the input slightly
            pred_batch = copy.deepcopy(batch)
            
            # Modify surface variables (simulate prediction)
            for var_name, tensor in pred_batch.surf_vars.items():
                # Remove history dimension (Aurora output has t=1)
                if tensor.dim() == 4:  # (b, t, h, w)
                    # Take last time step and add some random variation
                    last_step = tensor[:, -1:, :, :].clone()
                    
                    if var_name == '2t':  # Temperature - add small daily cycle
                        pred_batch.surf_vars[var_name] = last_step + torch.randn_like(last_step) * 0.5
                    elif var_name in ['10u', '10v']:  # Wind - add variability
                        pred_batch.surf_vars[var_name] = last_step + torch.randn_like(last_step) * 0.2
                    elif var_name == 'msl':  # Pressure - small changes
                        pred_batch.surf_vars[var_name] = last_step + torch.randn_like(last_step) * 50
                    else:
                        pred_batch.surf_vars[var_name] = last_step + torch.randn_like(last_step) * 0.1
            
            # Modify atmospheric variables
            for var_name, tensor in pred_batch.atmos_vars.items():
                if tensor.dim() == 5:  # (b, t, c, h, w)
                    last_step = tensor[:, -1:, :, :, :].clone()
                    pred_batch.atmos_vars[var_name] = last_step + torch.randn_like(last_step) * 0.1
            
            # Update metadata time (6 hours ahead)
            import datetime as dt
            current_time = pred_batch.metadata.time[0]
            next_time = current_time + dt.timedelta(hours=6)
            pred_batch.metadata.time = (next_time,)
            
            return pred_batch
        
        def __call__(self, batch):
            return self.forward(batch)
    
    return MockAurora()

# Load the model
aurora_model, is_real_model = load_aurora_model()

print(f"\n🎯 Model Status:")
print(f"   Model Type: {'Real Aurora' if is_real_model else 'Mock Demo'}")
print(f"   Ready for Predictions: ✅")

if not is_real_model:
    print(f"\n📝 Note: Using mock model for demonstration.")
    print(f"   To use real Aurora model, install from:")
    print(f"   https://github.com/microsoft/aurora")

In [ ]:
# Generate weather predictions using Aurora
print("🔮 Generating weather predictions for Maharashtra...")

def generate_predictions(model, initial_batch, num_steps=4):
    """Generate multiple time step predictions"""
    predictions = []
    current_batch = initial_batch
    
    print(f"   🎯 Generating {num_steps} prediction steps (6-hour intervals)")
    print(f"   ⏰ Starting from: {current_batch.metadata.time[0]}")
    
    for step in range(num_steps):
        print(f"      Step {step + 1}/{num_steps}...", end=" ")
        
        # Generate prediction
        with torch.no_grad():  # No gradients needed for inference
            pred_batch = model(current_batch)
        
        predictions.append(pred_batch)
        
        # For next iteration, use prediction as input
        # Aurora needs current + previous, so we combine them
        if hasattr(pred_batch, 'surf_vars') and hasattr(current_batch, 'surf_vars'):
            # Create new batch with prediction as current and old current as previous
            new_surf_vars = {}
            for var_name in pred_batch.surf_vars.keys():
                if var_name in current_batch.surf_vars:
                    # Combine: [previous, current] -> [old_current, prediction]
                    old_current = current_batch.surf_vars[var_name][:, -1:, :, :]  # Last time step
                    new_prediction = pred_batch.surf_vars[var_name]  # Should be shape (b, 1, h, w)
                    new_surf_vars[var_name] = torch.cat([old_current, new_prediction], dim=1)
            
            # Similar for atmospheric variables
            new_atmos_vars = {}
            for var_name in pred_batch.atmos_vars.keys():
                if var_name in current_batch.atmos_vars:
                    old_current = current_batch.atmos_vars[var_name][:, -1:, :, :, :]
                    new_prediction = pred_batch.atmos_vars[var_name]
                    new_atmos_vars[var_name] = torch.cat([old_current, new_prediction], dim=1)
            
            # Create new batch for next iteration
            current_batch = Batch(
                surf_vars=new_surf_vars,
                static_vars=current_batch.static_vars,  # Static vars don't change
                atmos_vars=new_atmos_vars,
                metadata=pred_batch.metadata
            )
        
        print(f"✓ {pred_batch.metadata.time[0]}")
    
    print(f"\n✅ Generated {len(predictions)} weather predictions!")
    return predictions

# Generate predictions
try:
    weather_predictions = generate_predictions(aurora_model, aurora_batch, num_steps=4)
    
    print(f"\n📊 Prediction Summary:")
    for i, pred in enumerate(weather_predictions):
        pred_time = pred.metadata.time[0]
        print(f"   Step {i+1}: {pred_time.strftime('%Y-%m-%d %H:%M UTC')}")
        
        # Show some statistics
        if '2t' in pred.surf_vars:
            temp_mean = pred.surf_vars['2t'].mean().item()
            temp_min = pred.surf_vars['2t'].min().item()
            temp_max = pred.surf_vars['2t'].max().item()
            print(f"            Temperature: {temp_mean:.1f}°C (range: {temp_min:.1f}-{temp_max:.1f}°C)")
        
        if 'msl' in pred.surf_vars:
            pressure_mean = pred.surf_vars['msl'].mean().item() / 100  # Convert to hPa
            print(f"            Pressure: {pressure_mean:.1f} hPa")

except Exception as e:
    print(f"❌ Error generating predictions: {e}")
    weather_predictions = []

## 7. Visualize Weather Predictions

Let's create comprehensive visualizations of the Aurora weather predictions for Maharashtra, including:
- Temperature evolution
- Wind pattern changes
- Pressure system movement
- Comparison with initial conditions

In [ ]:
# Visualize weather predictions
print("📊 Creating weather prediction visualizations...")

def extract_prediction_data(predictions):
    """Extract data from predictions for visualization"""
    pred_data = []
    
    for i, pred in enumerate(predictions):
        # Extract coordinates from metadata
        lats = pred.metadata.lat.numpy() if hasattr(pred.metadata.lat, 'numpy') else pred.metadata.lat
        lons = pred.metadata.lon.numpy() if hasattr(pred.metadata.lon, 'numpy') else pred.metadata.lon
        
        # Create coordinate meshgrid
        lon_grid, lat_grid = np.meshgrid(lons, lats)
        
        # Extract variables
        step_data = {
            'step': i + 1,
            'time': pred.metadata.time[0],
            'latitude': lat_grid,
            'longitude': lon_grid,
        }
        
        # Extract surface variables
        for var_name, tensor in pred.surf_vars.items():
            # tensor shape: (b, t, h, w) - take the current time step
            data = tensor[0, -1, :, :].numpy() if hasattr(tensor, 'numpy') else tensor[0, -1, :, :].detach().numpy()
            step_data[var_name] = data
        
        pred_data.append(step_data)
    
    return pred_data

# Extract prediction data
if weather_predictions:
    pred_data = extract_prediction_data(weather_predictions)
    
    print(f"✅ Extracted data for {len(pred_data)} prediction steps")
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(24, 18))
    
    # Temperature evolution
    print("   🌡️  Creating temperature evolution plots...")
    for i, data in enumerate(pred_data):
        ax = plt.subplot(3, 4, i + 1, projection=ccrs.PlateCarree())
        ax.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, alpha=0.8)
        ax.add_feature(cfeature.BORDERS, alpha=0.8)
        ax.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
        
        if '2t' in data:
            temp_data = data['2t'] - 273.15 if data['2t'].mean() > 100 else data['2t']  # Convert K to C if needed
            
            contour = ax.contourf(data['longitude'], data['latitude'], temp_data,
                                levels=20, cmap='RdYlBu_r', transform=ccrs.PlateCarree(),
                                extend='both')
            
            # Add wind vectors if available
            if '10u' in data and '10v' in data:
                # Subsample for cleaner arrows
                skip = 2
                ax.quiver(data['longitude'][::skip, ::skip], data['latitude'][::skip, ::skip],
                         data['10u'][::skip, ::skip], data['10v'][::skip, ::skip],
                         transform=ccrs.PlateCarree(), alpha=0.7, scale=50, width=0.003)
        
        ax.set_title(f'Step {data["step"]}: {data["time"].strftime("%Y-%m-%d %H:%M")}\nTemperature & Wind',
                    fontsize=10)
        ax.gridlines(draw_labels=True, alpha=0.3, linewidth=0.5)
        
        # Add colorbar for first plot
        if i == 0 and '2t' in data:
            plt.colorbar(contour, ax=ax, shrink=0.8, label='Temperature (°C)')
    
    # Wind speed evolution
    print("   💨 Creating wind speed evolution plots...")
    for i, data in enumerate(pred_data):
        ax = plt.subplot(3, 4, i + 5, projection=ccrs.PlateCarree())
        ax.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, alpha=0.8)
        ax.add_feature(cfeature.BORDERS, alpha=0.8)
        ax.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
        
        if '10u' in data and '10v' in data:
            wind_speed = np.sqrt(data['10u']**2 + data['10v']**2)
            
            contour = ax.contourf(data['longitude'], data['latitude'], wind_speed,
                                levels=15, cmap='viridis', transform=ccrs.PlateCarree(),
                                extend='max')
            
            # Add streamlines
            ax.streamplot(data['longitude'], data['latitude'],
                         data['10u'], data['10v'],
                         transform=ccrs.PlateCarree(), density=1, linewidth=0.8,
                         color='white', alpha=0.6)
        
        ax.set_title(f'Step {data["step"]}: Wind Speed & Direction', fontsize=10)
        ax.gridlines(draw_labels=True, alpha=0.3, linewidth=0.5)
        
        # Add colorbar for first plot
        if i == 0 and '10u' in data and '10v' in data:
            plt.colorbar(contour, ax=ax, shrink=0.8, label='Wind Speed (m/s)')
    
    # Pressure evolution
    print("   🌀 Creating pressure evolution plots...")
    for i, data in enumerate(pred_data):
        ax = plt.subplot(3, 4, i + 9, projection=ccrs.PlateCarree())
        ax.set_extent([72, 81, 15, 23], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, alpha=0.8)
        ax.add_feature(cfeature.BORDERS, alpha=0.8)
        ax.add_feature(cfeature.STATES, linestyle='--', alpha=0.5)
        
        if 'msl' in data:
            pressure_data = data['msl'] / 100  # Convert to hPa
            
            contour = ax.contour(data['longitude'], data['latitude'], pressure_data,
                               levels=15, colors='black', linewidths=1,
                               transform=ccrs.PlateCarree())
            ax.clabel(contour, inline=True, fontsize=8, fmt='%d')
            
            # Fill contours for better visualization
            contourf = ax.contourf(data['longitude'], data['latitude'], pressure_data,
                                 levels=15, cmap='RdBu_r', alpha=0.6,
                                 transform=ccrs.PlateCarree())
        
        ax.set_title(f'Step {data["step"]}: Sea Level Pressure', fontsize=10)
        ax.gridlines(draw_labels=True, alpha=0.3, linewidth=0.5)
        
        # Add colorbar for first plot
        if i == 0 and 'msl' in data:
            plt.colorbar(contourf, ax=ax, shrink=0.8, label='Pressure (hPa)')
    
    plt.tight_layout()
    plt.suptitle('Microsoft Aurora Weather Predictions for Maharashtra, India\n'
                 f'Forecast Period: {pred_data[0]["time"].strftime("%Y-%m-%d %H:%M")} to '
                 f'{pred_data[-1]["time"].strftime("%Y-%m-%d %H:%M")} UTC', 
                 fontsize=16, y=0.98)
    plt.show()
    
else:
    print("⚠️  No predictions available for visualization")

print("✅ Weather prediction visualization completed!")

In [ ]:
# Create time series analysis of predictions
print("📈 Creating time series analysis of weather predictions...")

if weather_predictions and pred_data:
    # Extract regional averages for time series
    times = [data['time'] for data in pred_data]
    
    # Calculate regional averages
    temp_avg = []
    pressure_avg = []
    wind_speed_avg = []
    
    for data in pred_data:
        if '2t' in data:
            temp_data = data['2t'] - 273.15 if data['2t'].mean() > 100 else data['2t']
            temp_avg.append(temp_data.mean())
        
        if 'msl' in data:
            pressure_avg.append(data['msl'].mean() / 100)  # Convert to hPa
        
        if '10u' in data and '10v' in data:
            wind_speed = np.sqrt(data['10u']**2 + data['10v']**2)
            wind_speed_avg.append(wind_speed.mean())
    
    # Create time series plots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Temperature time series
    if temp_avg:
        axes[0, 0].plot(times, temp_avg, 'r-o', linewidth=2, markersize=8, label='Average Temperature')
        axes[0, 0].set_ylabel('Temperature (°C)', fontsize=12)
        axes[0, 0].set_title('Temperature Evolution Over Maharashtra', fontsize=14, fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()
        
        # Add temperature range
        temp_min = [data['2t'].min() - 273.15 if data['2t'].mean() > 100 else data['2t'].min() for data in pred_data if '2t' in data]
        temp_max = [data['2t'].max() - 273.15 if data['2t'].mean() > 100 else data['2t'].max() for data in pred_data if '2t' in data]
        if temp_min and temp_max:
            axes[0, 0].fill_between(times, temp_min, temp_max, alpha=0.3, color='red', label='Temperature Range')
    
    # Pressure time series
    if pressure_avg:
        axes[0, 1].plot(times, pressure_avg, 'b-o', linewidth=2, markersize=8, label='Average Pressure')
        axes[0, 1].set_ylabel('Pressure (hPa)', fontsize=12)
        axes[0, 1].set_title('Pressure Evolution Over Maharashtra', fontsize=14, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()
    
    # Wind speed time series
    if wind_speed_avg:
        axes[1, 0].plot(times, wind_speed_avg, 'g-o', linewidth=2, markersize=8, label='Average Wind Speed')
        axes[1, 0].set_ylabel('Wind Speed (m/s)', fontsize=12)
        axes[1, 0].set_title('Wind Speed Evolution Over Maharashtra', fontsize=14, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].legend()
    
    # Summary statistics
    axes[1, 1].axis('off')
    
    # Create summary text
    summary_text = "🌤️ WEATHER FORECAST SUMMARY FOR MAHARASHTRA\n\n"
    
    if temp_avg:
        temp_change = temp_avg[-1] - temp_avg[0]
        summary_text += f"🌡️ TEMPERATURE:\n"
        summary_text += f"   Initial: {temp_avg[0]:.1f}°C\n"
        summary_text += f"   Final: {temp_avg[-1]:.1f}°C\n"
        summary_text += f"   Change: {temp_change:+.1f}°C\n\n"
    
    if pressure_avg:
        pressure_change = pressure_avg[-1] - pressure_avg[0]
        summary_text += f"🌀 PRESSURE:\n"
        summary_text += f"   Initial: {pressure_avg[0]:.1f} hPa\n"
        summary_text += f"   Final: {pressure_avg[-1]:.1f} hPa\n"
        summary_text += f"   Change: {pressure_change:+.1f} hPa\n\n"
    
    if wind_speed_avg:
        wind_change = wind_speed_avg[-1] - wind_speed_avg[0]
        summary_text += f"💨 WIND SPEED:\n"
        summary_text += f"   Initial: {wind_speed_avg[0]:.1f} m/s\n"
        summary_text += f"   Final: {wind_speed_avg[-1]:.1f} m/s\n"
        summary_text += f"   Change: {wind_change:+.1f} m/s\n\n"
    
    summary_text += f"📅 FORECAST PERIOD:\n"
    summary_text += f"   From: {times[0].strftime('%Y-%m-%d %H:%M UTC')}\n"
    summary_text += f"   To: {times[-1].strftime('%Y-%m-%d %H:%M UTC')}\n"
    summary_text += f"   Duration: {len(times)} × 6-hour steps\n\n"
    
    summary_text += f"🤖 MODEL: Microsoft Aurora\n"
    summary_text += f"📊 GRID POINTS: ~{pred_data[0]['latitude'].size}\n"
    summary_text += f"🗺️ REGION: Maharashtra, India"
    
    axes[1, 1].text(0.05, 0.95, summary_text, transform=axes[1, 1].transAxes,
                   fontsize=11, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
    
    # Format x-axis labels
    for ax in axes.flat:
        if ax != axes[1, 1]:  # Skip the summary panel
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.suptitle('Weather Prediction Time Series Analysis\nMaharashtra, India - Microsoft Aurora Model',
                fontsize=16, fontweight='bold', y=0.98)
    plt.show()
    
    print("✅ Time series analysis completed!")
    
else:
    print("⚠️  No prediction data available for time series analysis")

## 8. Model Performance Analysis

Let's analyze the Aurora model's predictions and provide insights about the weather forecast for Maharashtra.

In [ ]:
# Analyze model performance and provide weather insights
print("🔍 Analyzing Aurora model predictions and weather patterns...")

def analyze_weather_patterns(pred_data):
    """Analyze weather patterns from predictions"""
    analysis = {
        'temporal_trends': {},
        'spatial_patterns': {},
        'meteorological_insights': {},
        'extreme_events': {}
    }
    
    if not pred_data:
        return analysis
    
    # Temporal trends analysis
    print("   📈 Analyzing temporal trends...")
    
    times = [data['time'] for data in pred_data]
    
    if '2t' in pred_data[0]:
        temps = [data['2t'].mean() for data in pred_data]
        temps = [t - 273.15 if t > 100 else t for t in temps]  # Convert K to C if needed
        
        analysis['temporal_trends']['temperature'] = {
            'initial': temps[0],
            'final': temps[-1],
            'change': temps[-1] - temps[0],
            'trend': 'warming' if temps[-1] > temps[0] else 'cooling',
            'variability': np.std(temps)
        }
    
    if 'msl' in pred_data[0]:
        pressures = [data['msl'].mean() / 100 for data in pred_data]  # Convert to hPa
        
        analysis['temporal_trends']['pressure'] = {
            'initial': pressures[0],
            'final': pressures[-1],
            'change': pressures[-1] - pressures[0],
            'trend': 'rising' if pressures[-1] > pressures[0] else 'falling',
            'variability': np.std(pressures)
        }
    
    # Spatial patterns analysis
    print("   🗺️  Analyzing spatial patterns...")
    
    for i, data in enumerate(pred_data):
        if '2t' in data:
            temp_data = data['2t'] - 273.15 if data['2t'].mean() > 100 else data['2t']
            
            analysis['spatial_patterns'][f'step_{i+1}'] = {
                'temp_gradient': {
                    'north_south': temp_data[0, :].mean() - temp_data[-1, :].mean(),
                    'east_west': temp_data[:, -1].mean() - temp_data[:, 0].mean(),
                    'max_temp_location': np.unravel_index(temp_data.argmax(), temp_data.shape),
                    'min_temp_location': np.unravel_index(temp_data.argmin(), temp_data.shape)
                }
            }
        
        if '10u' in data and '10v' in data:
            wind_speed = np.sqrt(data['10u']**2 + data['10v']**2)
            wind_direction = np.arctan2(data['10v'], data['10u']) * 180 / np.pi
            
            if f'step_{i+1}' not in analysis['spatial_patterns']:
                analysis['spatial_patterns'][f'step_{i+1}'] = {}
            
            analysis['spatial_patterns'][f'step_{i+1}']['wind'] = {
                'max_wind_speed': wind_speed.max(),
                'avg_wind_speed': wind_speed.mean(),
                'dominant_direction': np.median(wind_direction),
                'max_wind_location': np.unravel_index(wind_speed.argmax(), wind_speed.shape)
            }
    
    # Meteorological insights
    print("   🌤️  Generating meteorological insights...")
    
    insights = []
    
    if 'temperature' in analysis['temporal_trends']:
        temp_trend = analysis['temporal_trends']['temperature']
        if abs(temp_trend['change']) > 2:
            insights.append(f"Significant temperature {temp_trend['trend']} of {temp_trend['change']:.1f}°C expected")
        
        if temp_trend['initial'] > 35:
            insights.append("Hot weather conditions with temperatures above 35°C")
        elif temp_trend['initial'] < 15:
            insights.append("Cool weather conditions with temperatures below 15°C")
    
    if 'pressure' in analysis['temporal_trends']:
        pressure_trend = analysis['temporal_trends']['pressure']
        if abs(pressure_trend['change']) > 5:
            insights.append(f"Notable pressure {pressure_trend['trend']} of {pressure_trend['change']:.1f} hPa")
        
        if pressure_trend['initial'] < 1010:
            insights.append("Low pressure system present - possible weather disturbance")
        elif pressure_trend['initial'] > 1020:
            insights.append("High pressure system - generally stable weather expected")
    
    analysis['meteorological_insights']['summary'] = insights
    
    return analysis

# Perform analysis
if weather_predictions and pred_data:
    weather_analysis = analyze_weather_patterns(pred_data)
    
    print("\n📊 WEATHER ANALYSIS RESULTS:")
    print("=" * 50)
    
    # Display temporal trends
    if weather_analysis['temporal_trends']:
        print("\n🕒 TEMPORAL TRENDS:")
        
        if 'temperature' in weather_analysis['temporal_trends']:
            temp = weather_analysis['temporal_trends']['temperature']
            print(f"   🌡️  Temperature: {temp['initial']:.1f}°C → {temp['final']:.1f}°C ({temp['change']:+.1f}°C, {temp['trend']})")
            print(f"       Variability: ±{temp['variability']:.1f}°C")
        
        if 'pressure' in weather_analysis['temporal_trends']:
            pres = weather_analysis['temporal_trends']['pressure']
            print(f"   🌀 Pressure: {pres['initial']:.1f} hPa → {pres['final']:.1f} hPa ({pres['change']:+.1f} hPa, {pres['trend']})")
            print(f"       Variability: ±{pres['variability']:.1f} hPa")
    
    # Display meteorological insights
    if weather_analysis['meteorological_insights']['summary']:
        print("\n🌤️  METEOROLOGICAL INSIGHTS:")
        for insight in weather_analysis['meteorological_insights']['summary']:
            print(f"   • {insight}")
    
    # Weather forecast summary
    print("\n📋 FORECAST SUMMARY FOR MAHARASHTRA:")
    print("-" * 40)
    
    if 'temperature' in weather_analysis['temporal_trends']:
        temp_trend = weather_analysis['temporal_trends']['temperature']
        if temp_trend['change'] > 1:
            print(f"🔥 Warming trend expected (+{temp_trend['change']:.1f}°C)")
        elif temp_trend['change'] < -1:
            print(f"❄️  Cooling trend expected ({temp_trend['change']:.1f}°C)")
        else:
            print(f"🌡️  Stable temperatures (~{temp_trend['initial']:.1f}°C)")
    
    if 'pressure' in weather_analysis['temporal_trends']:
        pres_trend = weather_analysis['temporal_trends']['pressure']
        if pres_trend['change'] > 2:
            print(f"📈 Rising pressure - improving weather")
        elif pres_trend['change'] < -2:
            print(f"📉 Falling pressure - possible weather changes")
        else:
            print(f"⚖️  Stable pressure conditions")
    
    print(f"\n⏱️  Forecast valid for: {len(pred_data)} × 6-hour periods")
    print(f"🤖 Generated by: Microsoft Aurora Model")
    print(f"📍 Region: Maharashtra, India")
    
else:
    print("⚠️  No prediction data available for analysis")

print("\n✅ Weather analysis completed!")

## 9. Export Results and Summary

Finally, let's export our results and create a comprehensive summary of the weather prediction workflow.

In [ ]:
# Export results and create summary
print("💾 Exporting results and creating workflow summary...")

import json
from datetime import datetime

# Create results directory
results_dir = f"../results_{config['request_id']}"
os.makedirs(results_dir, exist_ok=True)

# Export prediction data
if weather_predictions and pred_data:
    print("   📊 Exporting prediction data...")
    
    # Convert predictions to exportable format
    export_data = []
    
    for data in pred_data:
        # Convert numpy arrays to lists for JSON serialization
        export_step = {
            'step': data['step'],
            'time': data['time'].isoformat(),
            'variables': {}
        }
        
        # Extract grid data
        lats = data['latitude']
        lons = data['longitude']
        
        for var_name in ['2t', '10u', '10v', 'msl']:
            if var_name in data:
                var_data = data[var_name]
                # Convert to grid points
                points = []
                for i in range(var_data.shape[0]):
                    for j in range(var_data.shape[1]):
                        points.append({
                            'latitude': float(lats[i, j]),
                            'longitude': float(lons[i, j]),
                            'value': float(var_data[i, j])
                        })
                export_step['variables'][var_name] = points
        
        export_data.append(export_step)
    
    # Save predictions as JSON
    predictions_file = os.path.join(results_dir, 'aurora_predictions.json')
    with open(predictions_file, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"   ✅ Predictions saved to: {predictions_file}")

# Export weather analysis
if 'weather_analysis' in locals():
    print("   📈 Exporting weather analysis...")
    
    # Convert analysis to JSON-serializable format
    analysis_export = {}
    for key, value in weather_analysis.items():
        if isinstance(value, dict):
            analysis_export[key] = {}
            for subkey, subvalue in value.items():
                if isinstance(subvalue, (int, float, str, list)):
                    analysis_export[key][subkey] = subvalue
                elif isinstance(subvalue, dict):
                    analysis_export[key][subkey] = {k: v for k, v in subvalue.items() 
                                                  if isinstance(v, (int, float, str, list))}
    
    analysis_file = os.path.join(results_dir, 'weather_analysis.json')
    with open(analysis_file, 'w') as f:
        json.dump(analysis_export, f, indent=2)
    
    print(f"   ✅ Analysis saved to: {analysis_file}")

# Create comprehensive summary report
print("   📝 Creating summary report...")

summary_report = f"""
# ERA5 Data Processing and Microsoft Aurora Weather Prediction Report
**Region:** Maharashtra, India  
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}  
**Request ID:** {config['request_id']}  

## Executive Summary

This report presents the results of weather prediction for Maharashtra, India using:
- **Data Source:** ERA5 reanalysis data from ECMWF
- **Processing Tool:** eranest Python package
- **Prediction Model:** Microsoft Aurora AI weather model
- **Forecast Period:** {config['start_date'].strftime('%Y-%m-%d')} to {config['end_date'].strftime('%Y-%m-%d')}

## Data Processing Summary

### ERA5 Data Retrieved:
- **Surface Variables:** {', '.join(config['surface_variables'])}
- **Atmospheric Variables:** {', '.join(config['atmospheric_variables'])}
- **Static Variables:** {', '.join(config['static_variables'])}
- **Pressure Levels:** {', '.join(config['pressure_levels'])} hPa
- **Spatial Resolution:** {config['resolution']}° (~{config['resolution']*111:.0f}km)
- **Temporal Resolution:** {config['frequency']}

### Geographic Coverage:
- **State:** Maharashtra, India
- **Approximate Bounds:** 15.6°N to 22.0°N, 72.6°E to 80.9°E
- **Grid Points:** ~{pred_data[0]['latitude'].size if 'pred_data' in locals() and pred_data else 'N/A'}

## Weather Prediction Results
"""

if 'weather_analysis' in locals() and weather_analysis['temporal_trends']:
    if 'temperature' in weather_analysis['temporal_trends']:
        temp = weather_analysis['temporal_trends']['temperature']
        summary_report += f"""
### Temperature Forecast:
- **Initial Temperature:** {temp['initial']:.1f}°C
- **Final Temperature:** {temp['final']:.1f}°C
- **Temperature Trend:** {temp['trend'].capitalize()} ({temp['change']:+.1f}°C)
- **Temperature Variability:** ±{temp['variability']:.1f}°C
"""
    
    if 'pressure' in weather_analysis['temporal_trends']:
        pres = weather_analysis['temporal_trends']['pressure']
        summary_report += f"""
### Pressure Forecast:
- **Initial Pressure:** {pres['initial']:.1f} hPa
- **Final Pressure:** {pres['final']:.1f} hPa
- **Pressure Trend:** {pres['trend'].capitalize()} ({pres['change']:+.1f} hPa)
- **Pressure Variability:** ±{pres['variability']:.1f} hPa
"""
    
    if weather_analysis['meteorological_insights']['summary']:
        summary_report += f"""
### Key Meteorological Insights:
"""
        for insight in weather_analysis['meteorological_insights']['summary']:
            summary_report += f"- {insight}\n"

summary_report += f"""

## Technical Details

### Model Configuration:
- **Aurora Model Type:** {'Real' if is_real_model else 'Mock (Demonstration)'}
- **Prediction Steps:** {len(pred_data) if 'pred_data' in locals() and pred_data else 'N/A'}
- **Time Step:** 6 hours
- **Total Forecast Period:** {len(pred_data) * 6 if 'pred_data' in locals() and pred_data else 'N/A'} hours

### Data Quality:
- **ERA5 Data Status:** {'✅ Successfully Downloaded' if 'era5_data' in locals() else '⚠️ Mock Data Used'}
- **Aurora Processing:** {'✅ Completed' if 'aurora_batch' in locals() else '❌ Failed'}
- **Predictions Generated:** {'✅ Yes' if 'weather_predictions' in locals() and weather_predictions else '❌ No'}

## Files Generated:

1. **aurora_predictions.json** - Detailed prediction data
2. **weather_analysis.json** - Statistical analysis results
3. **summary_report.md** - This comprehensive report

## Usage Notes:

This demonstration showcases the complete workflow from ERA5 data retrieval to Aurora predictions. 
For operational use:

1. Ensure CDS API credentials are properly configured
2. Install Microsoft Aurora from the official repository
3. Consider computational resources for larger domains
4. Validate predictions against observations for accuracy assessment

---
**Generated by:** eranest package v0.1.0  
**Aurora Model:** Microsoft Aurora AI Weather Prediction  
**Data Source:** ECMWF ERA5 Reanalysis  
"""

# Save summary report
summary_file = os.path.join(results_dir, 'summary_report.md')
with open(summary_file, 'w') as f:
    f.write(summary_report)

print(f"   ✅ Summary report saved to: {summary_file}")

# Display final summary
print("\n" + "="*70)
print("🎉 WORKFLOW COMPLETED SUCCESSFULLY!")
print("="*70)
print(f"📁 Results Directory: {results_dir}")
print(f"📊 Predictions Generated: {len(pred_data) if 'pred_data' in locals() and pred_data else 0} time steps")
print(f"🌍 Region Covered: Maharashtra, India")
print(f"🤖 Model Used: Microsoft Aurora ({'Real' if is_real_model else 'Mock Demo'})")
print(f"📈 Analysis Completed: {'✅' if 'weather_analysis' in locals() else '❌'}")
print(f"💾 Files Exported: {'✅' if os.path.exists(summary_file) else '❌'}")

print("\n🔗 Next Steps:")
print("   1. Review the generated visualizations above")
print("   2. Check the exported files in the results directory")
print("   3. Validate predictions against observations (if available)")
print("   4. Consider extending the forecast period or domain")
print("   5. Explore different meteorological variables")

print(f"\n📚 For more information about eranest: https://github.com/JaggeryArray/eranest")
print(f"🤖 For Microsoft Aurora: https://github.com/microsoft/aurora")

print("\n" + "="*70)